In [252]:
import pandas as pd
from tabulate import tabulate

In [190]:
df = pd.read_csv('QUESTIONÁRIO ONLINE: Investigando a Percepção dos Desenvolvedores Brasileiros de Software  sobre Privacidade de Dados no uso de  LLMs.csv')

In [191]:
df.shape

(73, 44)

In [192]:
df.columns

Index(['Carimbo de data/hora', 'Você concorda em participar desta pesquisa?',
       '1. Qual a sua idade?', '2. Em qual Estado você mora?',
       '3. Qual seu nível de escolaridade?',
       '4. Qual é a sua principal função atualmente na área de desenvolvimento de software?',
       '5. Quantos anos de experiência você tem trabalhando em desenvolvimento de software?',
       '6. Qual seu nível de conhecimento sobre privacidade de dados?',
       '7. Você já participou de treinamentos ou cursos sobre privacidade de dados? ',
       '8. Qual sua principal fonte de informação sobre privacidade de dados? ',
       '9.  Qual seu nível de conhecimento sobre a Lei Geral de Proteção de Dados (LGPD)? ',
       '10. Qual seu nível de conhecimento sobre os princípios da Lei Geral de Proteção de Dados (LGPD)? ',
       '11. Quais são os principais princípios da LGPD que você conhece? (Marque todas as opções que se aplicam)',
       '12. Quais dos seguintes conceitos de proteção de dados da LGPD

In [359]:
idade_sts = df['1. Qual a sua idade?'].value_counts().to_frame()
idade_sts = idade_sts.sort_index(ascending=True)
idade_sts = idade_sts.reset_index(drop=False)
idade_sts = idade_sts.rename(columns={'index': 'FaixaEtaria'})

# Criando um dicionário para mapear as regioes para valores numéricos
dicionario_mapeamento = {'Entre 26 e 30 anos': 1,
                         'Entre 31 e 36 anos': 2,
                         'Entre 37 e 42 anos': 3,
                         'Entre 43 e 47 anos': 4,
                         "Entre 48 e 54 anos":5,
                         "Entre 55 a 60 anos":6}
# Criando uma nova coluna chamada 'ord_regiao' para posterior ordenacao por ela
idade_sts['ord_faixa_etaria'] = idade_sts['FaixaEtaria'].map(dicionario_mapeamento)

idade_sts = idade_sts.sort_values(by='ord_faixa_etaria', ascending=True)
idade_sts.drop('ord_faixa_etaria', axis=1, inplace=True)
idade_sts = idade_sts.reset_index(drop=True)


idade_sts.rename(columns={'1. Qual a sua idade?': 'qtde'}, inplace=True)
idade_sts['pct'] = ((idade_sts['qtde']/df.shape[0])*100).round(1)

idade_sts['FaixaEtaria'] = idade_sts['FaixaEtaria'].replace(["Entre 26 e 30 anos", 
                                                                         "Entre 31 e 36 anos",
                                                                         "Entre 37 e 42 anos",
                                                                         "Entre 43 e 47 anos",
                                                                         "Entre 48 e 54 anos",
                                                                         "Entre 55 a 60 anos"],
                                                                         ["26 - 30",
                                                                          "31 - 36",
                                                                          "37 - 42",
                                                                          "43 - 47",
                                                                          "48 - 54",
                                                                          "55 - 30"])
idade_sts

,FaixaEtaria,qtde,pct
0,26 - 30,4,5.5
1,31 - 36,11,15.1
2,37 - 42,23,31.5
3,43 - 47,20,27.4
4,48 - 54,12,16.4
5,55 - 30,3,4.1


In [360]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(idade_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 26 - 30 &  4 &  5.5 \\
 31 - 36 & 11 & 15.1 \\
 37 - 42 & 23 & 31.5 \\
 43 - 47 & 20 & 27.4 \\
 48 - 54 & 12 & 16.4 \\
 55 - 30 &  3 &  4.1 \\
\hline
\end{tabular}


In [255]:
estados_regiao = {
    "AC": "Norte",
    "AL": "Nordeste",
    "AM": "Norte",
    "AP": "Norte",
    "BA": "Nordeste",
    "CE": "Nordeste",
    "DF": "Centro-Oeste",
    "ES": "Sudeste",
    "GO": "Centro-Oeste",
    "MA": "Nordeste",
    "MG": "Sudeste",
    "MS": "Centro-Oeste",
    "MT": "Centro-Oeste",
    "PA": "Norte",
    "PB": "Nordeste",
    "PE": "Nordeste",
    "PI": "Nordeste",
    "PR": "Sul",
    "RJ": "Sudeste",
    "RN": "Nordeste",
    "RS": "Sul",
    "RO": "Norte",
    "RR": "Norte",
    "SC": "Sul",
    "SE": "Nordeste",
    "SP": "Sudeste",
    "TO": "Norte",
    "Nenhum (moro fora do Brasil)":"Exterior"
}

In [355]:
estado_moradia = df["2. Em qual Estado você mora?"].to_frame()
estado_moradia["Região"] = estado_moradia['2. Em qual Estado você mora?'].map(estados_regiao)
estado_moradia_sts = estado_moradia["Região"].value_counts().to_frame()

estado_moradia_sts = estado_moradia_sts.reset_index(drop=False)
estado_moradia_sts = estado_moradia_sts.rename(columns={'index': 'regiao_brasil','Região':'qtde'})

estado_moradia_sts['pct'] = ((estado_moradia_sts['qtde']/df.shape[0])*100).round(1)

# Criando um dicionário para mapear as regioes para valores numéricos
dicionario_mapeamento = {'Norte': 1,
                         'Nordeste': 2,
                         'Centro-Oeste': 3,
                         'Sudeste': 4,
                         "Sul":5,
                         "Exterior":6}
# Criando uma nova coluna chamada 'ord_regiao' para posterior ordenacao por ela
estado_moradia_sts['ord_regiao'] = estado_moradia_sts['regiao_brasil'].map(dicionario_mapeamento)

estado_moradia_sts = estado_moradia_sts.sort_values(by='ord_regiao', ascending=True)
estado_moradia_sts.drop('ord_regiao', axis=1, inplace=True)
estado_moradia_sts = estado_moradia_sts.reset_index(drop=True)


estado_moradia_sts['regiao_brasil'] = estado_moradia_sts['regiao_brasil'].replace(["Centro-Oeste", 
                                                                         "Exterior",
                                                                         "Nordeste",
                                                                         "Norte",
                                                                         "Sudeste",
                                                                         "Sul"],
                                                                         ["Midwest",
                                                                          "Abroad",
                                                                          "Northeast",
                                                                          "North",
                                                                          "Southeast",
                                                                          "South"])
estado_moradia_sts

,regiao_brasil,qtde,pct
0,North,3,4.1
1,Northeast,1,1.4
2,Midwest,59,80.8
3,Southeast,6,8.2
4,South,3,4.1
5,Abroad,1,1.4


In [356]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(estado_moradia_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 North     &  3 &  4.1 \\
 Northeast &  1 &  1.4 \\
 Midwest   & 59 & 80.8 \\
 Southeast &  6 &  8.2 \\
 South     &  3 &  4.1 \\
 Abroad    &  1 &  1.4 \\
\hline
\end{tabular}


In [337]:
nivel_escolaridade_sts = df["3. Qual seu nível de escolaridade?"].value_counts().to_frame()
nivel_escolaridade_sts = nivel_escolaridade_sts.reset_index(drop=False)
nivel_escolaridade_sts = nivel_escolaridade_sts.rename(columns={'index': 'Escolaridade'})

# Criando um dicionário para mapear os níveis de escolaridade para valores numéricos
dicionario_mapeamento = {'Graduado': 1,
                         'Pós-graduação Lato Sensu (Especialização) ou MBA': 2,
                         'Mestrado': 3,
                         'Doutorado': 4}
# Criando uma nova coluna chamada 'ord_escolaridade' para posterior ordenacao por ela
nivel_escolaridade_sts['ord_escolaridade'] = nivel_escolaridade_sts['Escolaridade'].map(dicionario_mapeamento)


nivel_escolaridade_sts = nivel_escolaridade_sts.sort_values(by='ord_escolaridade', ascending=True)
nivel_escolaridade_sts.drop('ord_escolaridade', axis=1, inplace=True)
nivel_escolaridade_sts = nivel_escolaridade_sts.reset_index(drop=True)


nivel_escolaridade_sts.rename(columns={'3. Qual seu nível de escolaridade?': 'qtde'}, inplace=True)
nivel_escolaridade_sts['pct'] = ((nivel_escolaridade_sts['qtde']/df.shape[0])*100).round(1)

nivel_escolaridade_sts['Escolaridade'] = nivel_escolaridade_sts['Escolaridade'].replace(["Graduado", 
                                                                         "Pós-graduação Lato Sensu (Especialização) ou MBA",
                                                                         "Mestrado",
                                                                         "Doutorado"],
                                                                         ["Graduated",
                                                                          "Specialist",
                                                                          "Master",
                                                                          "PhD"])
nivel_escolaridade_sts

,Escolaridade,qtde,pct
0,Graduated,14,19.2
1,Specialist,42,57.5
2,Master,10,13.7
3,PhD,6,8.2


In [338]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(nivel_escolaridade_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 Graduated  & 14 & 19.2 \\
 Specialist & 42 & 57.5 \\
 Master     & 10 & 13.7 \\
 PhD        &  6 &  8.2 \\
\hline
\end{tabular}


In [363]:
perfil = df["4. Qual é a sua principal função atualmente na área de desenvolvimento de software?"].replace(["Teste", "AppSec","Administrador de Banco de Dados"], ["Analista de qualidade e/ou testador", "Analista de Segurança de TI","Administrador de Dados"])
perfil_sts = perfil.value_counts().to_frame()

perfil_sts = perfil_sts.reset_index(drop=False)
perfil_sts = perfil_sts.rename(columns={'index': 'perfil'})
perfil_sts.rename(columns={'4. Qual é a sua principal função atualmente na área de desenvolvimento de software?': 'qtde'}, inplace=True)

perfil_sts['pct'] = ((perfil_sts['qtde']/df.shape[0])*100).round(1)

perfil_sts['perfil'] = perfil_sts['perfil'].replace(["Administrador de Dados",
                                                                "Analista de Segurança de TI",
                                                                "Analista de qualidade e/ou testador",
                                                                "Analista de requisitos",
                                                                "Arquiteto de Software",
                                                                "Dono do Produto (Product Owner)",
                                                                "Engenheiro DevOps",
                                                                "Engenheiro e/ou analista de dados",
                                                                "Gerente de projeto",
                                                                "Líder técnico e/ou de equipe",
                                                                "Programador e/ou desenvolvedor"],
                                                                ["Data Administrator",
                                                                "IT Security Analyst",
                                                                "Quality Analyst and/or Tester",
                                                                "Software Requirements Analyst",
                                                                "Software Architect",
                                                                "Product Owner",
                                                                "DevOps Engineer",
                                                                "Data Engineer and/or Analyst",
                                                                "Project Manager",
                                                                "Technical and/or Team Leader",
                                                                "Programmer and/or Developer"])

perfil_sts = perfil_sts.sort_values(by='perfil', ascending=True)
perfil_sts = perfil_sts.reset_index(drop=True)

perfil_sts

,perfil,qtde,pct
0,Data Administrator,1,1.4
1,Data Engineer and/or Analyst,6,8.2
2,DevOps Engineer,5,6.8
3,IT Security Analyst,2,2.7
4,Product Owner,2,2.7
5,Programmer and/or Developer,20,27.4
6,Project Manager,9,12.3
7,Quality Analyst and/or Tester,6,8.2
8,Scrum Master,3,4.1
9,Software Architect,7,9.6


In [364]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(perfil_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 Data Administrator            &  1 &  1.4 \\
 Data Engineer and/or Analyst  &  6 &  8.2 \\
 DevOps Engineer               &  5 &  6.8 \\
 IT Security Analyst           &  2 &  2.7 \\
 Product Owner                 &  2 &  2.7 \\
 Programmer and/or Developer   & 20 & 27.4 \\
 Project Manager               &  9 & 12.3 \\
 Quality Analyst and/or Tester &  6 &  8.2 \\
 Scrum Master                  &  3 &  4.1 \\
 Software Architect            &  7 &  9.6 \\
 Software Requirements Analyst &  2 &  2.7 \\
 Technical and/or Team Leader  & 10 & 13.7 \\
\hline
\end{tabular}


In [349]:
experiencia_sts = df["5. Quantos anos de experiência você tem trabalhando em desenvolvimento de software?"].value_counts().to_frame()

experiencia_sts = experiencia_sts.reset_index(drop=False)
experiencia_sts = experiencia_sts.rename(columns={'index': 'experiencia'})
experiencia_sts.rename(columns={'5. Quantos anos de experiência você tem trabalhando em desenvolvimento de software?': 'qtde'}, inplace=True)


# Criando um dicionário para mapear os níveis de escolaridade para valores numéricos
dicionario_mapeamento = {'Menos de 1 ano': 1,
                         'Entre 1 e 3 anos': 2,
                         'Entre 4 e 6 anos': 3,
                         'Entre 7 e 9 anos': 4,
                         'Entre 10 e 12 anos':5,
                         'Entre 13 e 15 anos':6,
                         'Mais de 15 anos':7
                         }
# Criando uma nova coluna chamada 'ord_experiencia' para posterior ordenacao por ela
experiencia_sts['ord_experiencia'] = experiencia_sts['experiencia'].map(dicionario_mapeamento)


experiencia_sts = experiencia_sts.sort_values(by='ord_experiencia', ascending=True)
experiencia_sts.drop('ord_experiencia', axis=1, inplace=True)
experiencia_sts = experiencia_sts.reset_index(drop=True)

experiencia_sts['pct'] = ((experiencia_sts['qtde']/df.shape[0])*100).round(1)

experiencia_sts['experiencia'] = experiencia_sts['experiencia'].replace(["Menos de 1 ano", 
                                                                         "Entre 1 e 3 anos",
                                                                         "Entre 4 e 6 anos",
                                                                         "Entre 7 e 9 anos",
                                                                         "Entre 10 e 12 anos",
                                                                         "Entre 13 e 15 anos",
                                                                         "Mais de 15 anos"],
                                                                         ["< 1",
                                                                          "1 - 3",
                                                                          "4 - 6",
                                                                          "7 - 9",
                                                                          "10 - 12",
                                                                          "13 - 15",
                                                                          "> 15"])

experiencia_sts

,experiencia,qtde,pct
0,< 1,2,2.7
1,1 - 3,5,6.8
2,4 - 6,5,6.8
3,7 - 9,6,8.2
4,10 - 12,10,13.7
5,13 - 15,6,8.2
6,> 15,39,53.4


In [350]:
# Criando a tabela LaTeX com o pacote `tabulate`
tabela_latex = tabulate(experiencia_sts, floatfmt=".1f", tablefmt="latex", showindex="False")

# Imprimindo a tabela LaTeX
print(tabela_latex)

\begin{tabular}{lrr}
\hline
 \ensuremath{<} 1     &  2 &  2.7 \\
 1 - 3   &  5 &  6.8 \\
 4 - 6   &  5 &  6.8 \\
 7 - 9   &  6 &  8.2 \\
 10 - 12 & 10 & 13.7 \\
 13 - 15 &  6 &  8.2 \\
 \ensuremath{>} 15    & 39 & 53.4 \\
\hline
\end{tabular}


In [191]:
conhecimento_privacidade = df["6. Qual seu nível de conhecimento sobre privacidade de dados?"].to_frame()
conhecimento_privacidade = conhecimento_privacidade["6. Qual seu nível de conhecimento sobre privacidade de dados?"].str.split('(', expand=True)[0]
conhecimento_privacidade.value_counts()


Conhecimento básico       37
Conhecimento bom          22
Conhecimento completo      7
Noções básicas             6
Name: 0, dtype: int64

In [192]:
conhecimento_privacidade = conhecimento_privacidade.to_frame()
conhecimento_privacidade['#'] = range(1,len(conhecimento_privacidade)+1)
conhecimento_privacidade = conhecimento_privacidade.set_index(['#'])
conhecimento_privacidade.rename(columns={0: 'Q6'}, inplace=True)
conhecimento_privacidade

,Q6
#,
1,Conhecimento completo
2,Conhecimento bom
3,Conhecimento bom
4,Conhecimento básico
5,Conhecimento básico
...,...
68,Noções básicas
69,Conhecimento bom
70,Conhecimento básico


In [96]:
conhecimento_privacidade = conhecimento_privacidade.to_frame()
conhecimento_privacidade.set_index(range(1, len(conhecimento_privacidade) + 1))


KeyError: 'None of [range(1, 73)] are in the columns'

In [101]:
conhecimento_privacidade.to_frame().reset_index()

,index,0
0,0,Conhecimento completo
1,1,Conhecimento bom
2,2,Conhecimento bom
3,3,Conhecimento básico
4,4,Conhecimento básico
...,...,...
67,67,Noções básicas
68,68,Conhecimento bom
69,69,Conhecimento básico
70,70,Conhecimento básico


In [79]:
conhecimento_privacidade_series = conhecimento_privacidade.reset_index(drop=True)
conhecimento_privacidade = conhecimento_privacidade_series.to_frame()

In [84]:
range(1, len(conhecimento_privacidade))

range(1, 72)

In [56]:
conhecimento_privacidade = pd.DataFrame(conhecimento_privacidade)
conhecimento_privacidade = conhecimento_privacidade.set_index(range(1, len(conhecimento_privacidade) + 1))
conhecimento_privacidade

KeyError: 'None of [range(1, 73)] are in the columns'

In [97]:
conhecimento_privacidade_2 = conhecimento_privacidade["6. Qual seu nível de conhecimento sobre privacidade de dados?"].replace(["Conhecimento bom", "Conhecimento completo"], ["Conhecimento intermediário", "Conhecimento avançado"])
conhecimento_privacidade_2.value_counts()
conhecimento_privacidade_2

KeyError: '6. Qual seu nível de conhecimento sobre privacidade de dados?'